In [ ]:
import pandas as pd

# Calculate Tenkan-sen
small_period_high = df["high"].rolling(window=small_period).max()
small_period_low = df["low"].rolling(window=small_period).min()
df["tenkan_sen"] = (small_period_high + small_period_low) / 2

# Calculate Kijun-sen
large_period_high = df["high"].rolling(window=large_period).max()
large_period_low = df["low"].rolling(window=large_period).min()
df["kijun_sen"] = (large_period_high + large_period_low) / 2

# Calculate Senkou Span A
df["senkou_span_a"] = (df["tenkan_sen"] + df["kijun_sen"]) / 2

# Calculate Senkou Span B
rolling_high = df["high"].rolling(window=rolling_period).max()
rolling_low = df["low"].rolling(window=rolling_period).min()
df["senkou_span_b"] = (rolling_high + rolling_low) / 2
df["senkou_span_b"] = df["senkou_span_b"].shift(40)

# Calculate smoothed close price
df["close_smooth"] = df["close"].rolling(window=5).mean()

# Calculate Average True Range (ATR)
df['tr0'] = abs(df["high"] - df["low"])
df['tr1'] = abs(df["high"] - df["close"].shift())
df['tr2'] = abs(df["low"] - df["close"].shift())
tr = df[['tr0', 'tr1', 'tr2']].max(axis=1)
df["ATR"] = tr.rolling(window=atr_period).mean()

# Calculate stop-loss thresholds
df['stop_loss_long'] = df["high"].rolling(22).max() - 2 * df["ATR"]
df['stop_loss_short'] = df["low"].rolling(22).min() + 2 * df["ATR"]

# Initialize signals column
df["signals"] = 0

# Generate long signals
long_condition = (
    (df['close_smooth'] < df['stop_loss_long']) &
    (df.index >= 22) &
    ((df['close'] < df['kijun_sen']) |
     (df['close'] < df['tenkan_sen'])) &
     ((df['close'] < df['senkou_span_a']) |
     (df['close'] < df['senkou_span_b'])) &
     (df['close'] < df['LowerBand'])
)
df.loc[long_condition, 'signals'] = -0.4  # Signal to exit the long trade

# Generate short signals
short_condition = (
    (df['close_smooth'] > df['stop_loss_short']) &
    (df.index >= 22) &
    ((df['close'] > df['kijun_sen']) |
     (df['close'] > df['tenkan_sen'])) &
     ((df['close'] > df['senkou_span_a']) |
     (df['close'] > df['senkou_span_b'])) &
     (df['close'] > df['UpperBand'])
)
df.loc[short_condition, 'signals'] = 0.4  # Signal to enter short trade

# Display the DataFrame with signals
df
